https://python.langchain.com/docs/use_cases/extraction/how_to/parse/

# Retrieval
Add context to the LLM

In [10]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("pdfs/A00041-00045.pdf")
loaded = loader.load()

In [11]:
type(loaded)

list

In [12]:
loaded[0]

Document(page_content='BOE núm. 1 Lunes 1 enero 2001 41\nIII. Otras disposiciones\nCORTES GENERALES\n24 ACUERDO de 19 de diciembre de 2000, de la Mesa del Sena-\ndo, referente a la convocatoria de becas de formación sobre\ncomunicación institucional relacionada con el Senado.\nEl Senado, por acuerdo de la Mesa de 19 diciembre de 2000, convoca\ntres becas individuales de formación sobre comunicación institucional rela-\ncionada con el Senado, de acuerdo con las bases que figuran a continuación:\nPrimera. Objeto.— El Senado convoca tres becas individuales para rea-\nlizar actividades de formación teórico-práctica sobre comunicación ins-titucional relacionada con la actividad parlamentaria que determine el\nDepartamento de Prensa de la Cámara.\nSegunda. Condiciones de las becas.— 1. Cada una de las tres becas\nse disfrutará desde su otorgamiento por la Mesa hasta el 31 de diciembre\nde 2001 y su cuantía mensual será de 107.083 pesetas brutas, que se\npercibirán finalizado cada mes.\n2. El

In [13]:
loaded[1]

Document(page_content='42 Lunes 1 enero 2001 BOE núm. 1\n', metadata={'source': 'pdfs/A00041-00045.pdf', 'page': 1})

In [14]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(loaded)

In [15]:
import time
start = time.time()
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OllamaEmbeddings())
end = time.time()
embedding_time = end - start
print("Time taken:", embedding_time)

Time taken: 29.40017795562744


Cuál es la cuantía mensual bruta de las becas de formación sobre comunicación institucional relacionada con el Senado

In [16]:
import os
len(os.listdir("pdfs"))

4313

In [17]:
print(f"El tiempo que tomaria hacer embeddings de todos los documentos es de {len(os.listdir('pdfs')) * embedding_time / 60} minutos que son {len(os.listdir('pdfs')) * embedding_time / 3600} horas")

El tiempo que tomaria hacer embeddings de todos los documentos es de 2113.382792043686 minutos que son 35.22304653406143 horas


In [18]:
# k is the number of chunks to retrieve
retriever = vectorstore.as_retriever(k=4)

docs = retriever.invoke("Cuál es la cuantía mensual bruta de las becas de formación sobre comunicación institucional relacionada con el Senado?")

docs

[Document(page_content='todo lo que se refiere a los asuntos, hechos o informaciones de los que\nhubiera tenido conocimiento durante el período de la beca. No deberán\ncomunicar bajo ninguna forma documentos o informaciones que no hubie-\nran sido hechos públicos a personas que no estuvieren cualificados para\ntener conocimientos de los mismos, a reserva del acuerdo previo de la\nInstitución. Seguirán sometidos a esta obligación después de finalizar su\nbeca.', metadata={'page': 0, 'source': 'pdfs/A00041-00045.pdf'}),
 Document(page_content='Primera. Objeto.— El Senado convoca tres becas individuales para rea-\nlizar actividades de formación teórico-práctica sobre comunicación ins-titucional relacionada con la actividad parlamentaria que determine el\nDepartamento de Prensa de la Cámara.\nSegunda. Condiciones de las becas.— 1. Cada una de las tres becas\nse disfrutará desde su otorgamiento por la Mesa hasta el 31 de diciembre\nde 2001 y su cuantía mensual será de 107.083 pesetas brutas

In [44]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2", format="json")

In [101]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# SYSTEM_TEMPLATE = """
# Answer the user's questions based on the below context. 
# If the context doesn't contain any relevant information to the question, don't make something up and just say "I don't know":

# The output must be a json object with the following structure:
# {
#     "cantidad": 1000,
#     "moneda": "EUR"
# }

# Lo anterior es un ejemplo. La key "cantidad" debe ser un número que no tiene por que ser 1000 y la key "moneda" debe ser un string que representa la moneda en la que se encuentra la cantidad.
# SOLO PUEDE HABER DOS KEYS EN EL JSON, "cantidad" y "moneda".
# <context>
# {context}
# </context>
# """

# SYSTEM_TEMPLATE = """
# Answer the user's questions based on the below context. 
# If the context doesn't contain any relevant information to the question, don't make something up and just say "I don't know":

# <context>
# {context}
# </context>
# """

SYSTEM_TEMPLATE = """
Answer the user's questions based on the below context. If the context doesn't contain any relevant information to the question, don't make something up and just say "I don't know".

Your task is to extract the amount of money.
If you answer with a json their must only be one key, "cantidad" which is a number and represents the amount of money.
<context>
{context}
</context>
"""

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            SYSTEM_TEMPLATE,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(llm, question_answering_prompt)

In [102]:
type(docs)
docs

[Document(page_content='todo lo que se refiere a los asuntos, hechos o informaciones de los que\nhubiera tenido conocimiento durante el período de la beca. No deberán\ncomunicar bajo ninguna forma documentos o informaciones que no hubie-\nran sido hechos públicos a personas que no estuvieren cualificados para\ntener conocimientos de los mismos, a reserva del acuerdo previo de la\nInstitución. Seguirán sometidos a esta obligación después de finalizar su\nbeca.', metadata={'page': 0, 'source': 'pdfs/A00041-00045.pdf'}),
 Document(page_content='Primera. Objeto.— El Senado convoca tres becas individuales para rea-\nlizar actividades de formación teórico-práctica sobre comunicación ins-titucional relacionada con la actividad parlamentaria que determine el\nDepartamento de Prensa de la Cámara.\nSegunda. Condiciones de las becas.— 1. Cada una de las tres becas\nse disfrutará desde su otorgamiento por la Mesa hasta el 31 de diciembre\nde 2001 y su cuantía mensual será de 107.083 pesetas brutas

In [103]:
from langchain_core.messages import HumanMessage

json = document_chain.invoke(
    {
        "context": docs,
        "messages": [
            HumanMessage(content="Cuál es la cuantía mensual bruta de las becas de formación sobre comunicación institucional relacionada con el Senado?")
        ],
    }
)
json

'{"cantidad": 107.083}'

In [95]:
import re

def extract_quantities(text):
    # This pattern matches sequences of digits that may have commas or periods for formatting
    # and are not surrounded by spaces, but could be next to text or punctuation
    quantities = re.findall(r'(?<!\s)[0-9,\.]+(?!\s)', text)
    # Remove commas or periods if they are used as thousands separators or decimal points
    return [quantity.replace(',', '').replace('.', '') for quantity in quantities if quantity.replace(',', '').replace('.', '').isdigit()]

# Example usage
text = "The price is 107,083pesetas and there are 300 students."
quantities = extract_quantities(text)
print(quantities)

print(extract_quantities(json))

['07083', '0']
['0708']


In [100]:
SYSTEM_TEMPLATE = """
Your task is to improve a json object with the following structure:
 {
     "cantidad": 1000,
     "moneda": "EUR"
 }

 <context>
 {context}
 </context>
 """

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            SYSTEM_TEMPLATE,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
document_chain = create_stuff_documents_chain(llm, question_answering_prompt)

In [89]:
document_chain.invoke(
    {
        "context": json,
        "messages": [
            HumanMessage(content="I don't know")
        ],
    }
)

AttributeError: 'str' object has no attribute 'page_content'

In [ ]:
docu

# Structured Data Extraction

In [53]:
from langchain.llms import Ollama
model = Ollama(model="llama2", temperature=0, format="json")

In [54]:
from typing import List, Optional

from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator


class Subvenvion(BaseModel):
    """Cantidad de la subvencion."""

    cantidad: float = Field(..., description="Amount subsidized.")



In [55]:
# Set up a parser
parser = PydanticOutputParser(pydantic_object=Subvenvion)

# Prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user query. Wrap the output in `json` tags\n{format_instructions}",
        ),
        ("human", "{query}"),
    ]
).partial(format_instructions=parser.get_format_instructions())

In [56]:
query = "¿Cuál es la cuantía mensual bruta de las becas de formación sobre comunicación institucional relacionada con el Senado?"

In [57]:
print(prompt.format_prompt(query=query).to_string())

System: Answer the user query. Wrap the output in `json` tags
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "Cantidad de la subvencion.", "properties": {"cantidad": {"title": "Cantidad", "description": "Amount subsidized.", "type": "number"}}, "required": ["cantidad"]}
```
Human: ¿Cuál es la cuantía mensual bruta de las becas de formación sobre comunicación institucional relacionada con el Senado?


In [58]:
chain = prompt | model | parser
chain.invoke({"query": query})

OutputParserException: Failed to parse Subvenvion from completion {"description": "Cantidad mensual bruta de las befas de formaci\u00f3n sobre comunicaci\u00f3n institucional relacionada con el Senado", "properties": {"cantidad": {"title": "Cantidad", "description": "Amount of the monthly subsidy for training on institutional communication related to the Senate", "type": "number"}}, "required": ["cantidad"]}. Got: 1 validation error for Subvenvion
cantidad
  field required (type=value_error.missing)